In [ ]:
from IPython.display import clear_output
import sys
!pip install mteb
!pip install -q transformers accelerate bitsandbytes
! pip install git+https://github.com/simonzhang00/ripser-plusplus.git
! pip install git+https://github.com/IlyaTrofimov/RTD.git
!git clone https://github.com/mklabunde/resi.git
# !cd resi && pip install -r requirements.txt && pip install -e .
!pip install loguru
!pip install git+https://github.com/KhrulkovV/geometry-score.git
!pip install git+https://github.com/xgfs/imd.git
!pip install gudhi
!pip install hf_xet
sys.path.append('/content/resi/')
clear_output()

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
from datasets import load_dataset
from tqdm.auto import tqdm
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import gc
import os
import cvxpy as cp


from typing import List
import mteb

import numpy as np
from tqdm.auto import tqdm
import json


from sklearn.decomposition import IncrementalPCA

In [ ]:
from repsim.measures import ALL_MEASURES
from repsim.measures import FUNCTIONAL_SIMILARITY_MEASURES

import pickle
def save_res(dct, path):
  with open(path, 'wb') as handle:
      pickle.dump(dct, handle, protocol=pickle.HIGHEST_PROTOCOL)

def load_res(path):
  try:
    with open(path, 'rb') as handle:
      results = pickle.load(handle)
  except:
    results = {}

  return results

2025-05-23 18:18:55.794 | INFO     | repsim.measures.rtd:__init__:56 - RTD will use cuda devices to compute barcodes. It is not possible to specify which GPU directly. Use the CUDA_VISIBLE_DEVICES environment variable to specify which GPU to use.
2025-05-23 18:18:55.795 | INFO     | repsim.measures.rtd:__init__:56 - RTD will use cuda devices to compute barcodes. It is not possible to specify which GPU directly. Use the CUDA_VISIBLE_DEVICES environment variable to specify which GPU to use.


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


# Считаем метрики похожести разных слоев модели CLS- токену + Pooler

Тексты для расчета

In [ ]:
"""
Generate CLS embeddings for every hidden layer **and** the SimCSE‑trained `pooler_output`,
then save them on disk batch‑by‑batch.
"""

# 30 000 short samples from C4 (demo size)
ds    = load_dataset("allenai/c4", "en", split="train", streaming=True)
texts = [ex["text"] for ex in ds.shuffle(buffer_size=30_000, seed=42).take(30_000)]

# =============================================================
# 1.  Configuration
# =============================================================
MODEL_NAME = "princeton-nlp/sup-simcse-roberta-base"  # *** SimCSE ***
SAVE_DIR   = "simcse_cls_embeddings"
os.makedirs(SAVE_DIR, exist_ok=True)

BATCH_SIZE = 256
MAX_LEN    = 256

# =============================================================
# 2.  Model & tokenizer
# =============================================================

device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Using device: {device}")

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

model = AutoModel.from_pretrained(
    MODEL_NAME,
    output_hidden_states=True,              # we need every layer
    device_map="auto" if device == "cuda" else None
).eval()

num_layers = model.config.num_hidden_layers + 1  # e.g. 13 for *-base (embeddings layer + 12 transformer layers)

# =============================================================
# 3.  Generation & saving of CLS + pooler embeddings
# =============================================================

problem_batches = []

for batch_idx in tqdm(range(0, len(texts), BATCH_SIZE),
                      total=(len(texts) + BATCH_SIZE - 1) // BATCH_SIZE):

    batch_texts = texts[batch_idx: batch_idx + BATCH_SIZE]

    inputs = tokenizer(
        batch_texts,
        padding=True,
        truncation=True,
        max_length=MAX_LEN,
        return_tensors="pt"
    ).to(model.device)

    with torch.no_grad():
        outputs = model(**inputs)  # hidden_states + pooler_output

    # ---------- CLS for every hidden layer ----------
    for layer_idx, hs in enumerate(outputs.hidden_states):  # hs: [B, L, H]
        cls_emb = hs[:, 0, :].cpu()                        # [B, H]

        if torch.isinf(cls_emb).any():
            problem_batches.append((batch_idx, f"layer_{layer_idx}"))

        layer_dir = os.path.join(SAVE_DIR, f"layer_{layer_idx}")
        os.makedirs(layer_dir, exist_ok=True)
        torch.save(cls_emb, os.path.join(layer_dir, f"batch_{batch_idx}.pt"))

    # ---------- SimCSE‑trained pooler_output ----------
    if hasattr(outputs, "pooler_output") and outputs.pooler_output is not None:
        pool_emb = outputs.pooler_output.cpu()  # [B, H]

        if torch.isinf(pool_emb).any():
            problem_batches.append((batch_idx, "pooler"))

        pooler_dir = os.path.join(SAVE_DIR, "pooler")
        os.makedirs(pooler_dir, exist_ok=True)
        torch.save(pool_emb, os.path.join(pooler_dir, f"batch_{batch_idx}.pt"))

    # ---------- housekeeping ----------
    del outputs, inputs, cls_emb
    if "pool_emb" in locals():
        del pool_emb
    gc.collect(); torch.cuda.empty_cache()

# =============================================================
# 4.  Summary
# =============================================================
if problem_batches:
    print("Found inf values in:", problem_batches)
print(f"All CLS and pooler embeddings saved under {SAVE_DIR}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/41.1k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/1024 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/1024 [00:00<?, ?it/s]

Using device: cuda


...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

  0%|          | 0/118 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

All CLS and pooler embeddings saved under simcse_cls_embeddings


In [ ]:
def load_layer_embeddings(layer_idx: int, num_layers = 13) -> torch.Tensor:
    """Return concatenated embeddings for `layer_idx`.
    The last index (== num_layers) denotes the pooler_output."""
    if layer_idx == num_layers:   # pooler
        layer_dir = os.path.join(SAVE_DIR, "pooler")
    else:
        layer_dir = os.path.join(SAVE_DIR, f"layer_{layer_idx}")

    batch_files = sorted(layer_dir for layer_dir in os.listdir(layer_dir))
    return torch.cat([torch.load(os.path.join(layer_dir, f)) for f in batch_files], dim=0)

In [ ]:
s = load_layer_embeddings(1)

In [ ]:
for layer_idx in range(num_layers+1):
  a = load_layer_embeddings(layer_idx)
  if torch.isinf(a).sum() != 0:
    print(f"INF layer, {layer_idx}")
  if torch.isnan(a).sum() != 0:
    print(f"NANS layer, {layer_idx}")

In [ ]:
import logging
from datetime import datetime
import numpy as np
import torch
import gc

# === Настройка логгера ===
def setup_logging(log_file):
    logger = logging.getLogger("metrics_logger")
    logger.setLevel(logging.INFO)

    if logger.hasHandlers():
        logger.handlers.clear()

    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

    file_handler = logging.FileHandler(log_file, mode='w')
    file_handler.setFormatter(formatter)

    console_handler = logging.StreamHandler()
    console_handler.setFormatter(formatter)

    logger.addHandler(file_handler)
    logger.addHandler(console_handler)

    return logger

# === Подсчет метрики ===
def calculate_metric(a, b, metric_name, batch_size=1000, trials=10):
    cls = ALL_MEASURES[metric_name]
    metrics = []

    for i in range(trials):
        idx = np.random.permutation(a.shape[0])[:batch_size]
        metric = cls(a[idx], b[idx], shape='nd')
        metrics.append(metric)

    return np.mean(metrics), np.std(metrics)

In [ ]:
PATH = '/content/drive/MyDrive/PTLS/NLP/SIM-SCE/'
LOG_FILE = 'similarity_metrics.log'

In [ ]:
# === Константы ===
METRICS = ['CKA', 'RSA', 'JaccardSimilarity', 'DistanceCorrelation']

# === Инициализация логгера ===
logger = setup_logging(LOG_FILE)

# === Основной процесс ===
try:
    logger.info("=== Start metrics calculation ===")
    logger.info(f"Metrics to compute: {', '.join(METRICS)}")
    start_time = datetime.now()

    for metric_name in METRICS:
        metric_start = datetime.now()
        logger.info(f"{'='*40}\nProcessing: {metric_name}")

        matrix = np.zeros((num_layers+1, num_layers+1))
        matrix_std = np.zeros((num_layers+1, num_layers+1))

        for i in range(num_layers+1):
            a = load_layer_embeddings(i)
            logger.info(f"Layer {i} | Shape: {a.shape}")

            for j in range(i, num_layers+1):
                b = load_layer_embeddings(j)

                try:
                    metr, metr_std = calculate_metric(a.float(), b.float(), metric_name, batch_size=3000, trials=5)
                    matrix[i, j] = matrix[j, i] = metr
                    matrix_std[i, j] = matrix_std[j, i] = metr_std

                    logger.info(f"{metric_name} | L{i}-L{j}: {metr:.4f} ± {metr_std:.4f} | "
                                f"GPU Mem: {torch.cuda.memory_allocated() / 1024**2:.1f} MB")
                except Exception as e:
                    logger.error(f"Metric error for L{i}-L{j}: {e}", exc_info=True)

                del b
                torch.cuda.empty_cache()
                gc.collect()
                clear_output()

            del a
            torch.cuda.empty_cache()
            gc.collect()

        save_res(matrix, f"{PATH}{metric_name}.pkl")
        save_res(matrix_std, f"{PATH}{metric_name}_std.pkl")

        logger.info(f"Finished {metric_name} in {datetime.now() - metric_start}")


except Exception as e:
    logger.critical(f"Fatal error: {e}", exc_info=True)
finally:
    logger.info(f"=== All metrics completed in {datetime.now() - start_time} ===")
    for handler in logger.handlers:
        handler.close()

2025-05-22 10:07:38,751 - INFO - Finished DistanceCorrelation in 0:04:34.121221
INFO:metrics_logger:Finished DistanceCorrelation in 0:04:34.121221
2025-05-22 10:07:38,752 - INFO - === All metrics completed in 0:20:33.686556 ===
INFO:metrics_logger:=== All metrics completed in 0:20:33.686556 ===


# Считаем метрики похожести разных слоев модели mean- токену + Pooler

Тексты для расчета

In [ ]:
"""
Generate MEAN-pooled embeddings for every hidden layer **and** the SimCSE-trained
`pooler_output`, then save them on disk batch-by-batch.
"""

import os, gc, torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel
from tqdm.auto import tqdm

# 30 000 short samples from C4 (demo size)
ds    = load_dataset("allenai/c4", "en", split="train", streaming=True)
texts = [ex["text"] for ex in ds.shuffle(buffer_size=30_000, seed=42).take(30_000)]

# =============================================================
# 1.  Configuration
# =============================================================
MODEL_NAME = "princeton-nlp/sup-simcse-roberta-base"  # *** SimCSE ***
SAVE_DIR   = "simcse_mean_embeddings"                 # ←- поменял имя директории
os.makedirs(SAVE_DIR, exist_ok=True)

BATCH_SIZE = 256
MAX_LEN    = 256

# =============================================================
# 2.  Model & tokenizer
# =============================================================

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

model = AutoModel.from_pretrained(
    MODEL_NAME,
    output_hidden_states=True,     # нужно всё
    device_map="auto" if device == "cuda" else None
).eval()

num_layers = model.config.num_hidden_layers + 1  # embeddings + transformer layers

# =============================================================
# 3.  Generation & saving of MEAN + pooler embeddings
# =============================================================

problem_batches = []

for batch_idx in tqdm(range(0, len(texts), BATCH_SIZE),
                      total=(len(texts) + BATCH_SIZE - 1) // BATCH_SIZE):

    batch_texts = texts[batch_idx: batch_idx + BATCH_SIZE]

    inputs = tokenizer(
        batch_texts,
        padding=True,
        truncation=True,
        max_length=MAX_LEN,
        return_tensors="pt"
    ).to(model.device)

    with torch.no_grad():
        outputs = model(**inputs)  # hidden_states + pooler_output

    # ---------- MEAN over tokens for every hidden layer ----------
# ---------- MEAN over non-padding tokens for every hidden layer ----------
    attn_mask = inputs.attention_mask                     # [B, L], 1-для-токенов, 0-для-PAD
    lengths   = attn_mask.sum(dim=1, keepdim=True)        # [B, 1]

    # избегаем деления на 0 (может случиться, если строка обрезана целиком)
    lengths = lengths.clamp(min=1)

    # цикл по слоям
    for layer_idx, hs in enumerate(outputs.hidden_states):   # hs: [B, L, H]
        # обнуляем PAD-позиции и делим только на число непаддингов
        summed = (hs * attn_mask.unsqueeze(-1)).sum(dim=1)   # [B, H]
        mean_emb = summed / lengths                          # [B, H]

        if torch.isinf(mean_emb).any():
            problem_batches.append((batch_idx, f"layer_{layer_idx}"))

        layer_dir = os.path.join(SAVE_DIR, f"layer_{layer_idx}")
        os.makedirs(layer_dir, exist_ok=True)
        torch.save(mean_emb.cpu(), os.path.join(layer_dir, f"batch_{batch_idx}.pt"))


    # ---------- SimCSE-trained pooler_output ----------
    if hasattr(outputs, "pooler_output") and outputs.pooler_output is not None:
        pool_emb = outputs.pooler_output.cpu()  # [B, H]

        if torch.isinf(pool_emb).any():
            problem_batches.append((batch_idx, "pooler"))

        pooler_dir = os.path.join(SAVE_DIR, "pooler")
        os.makedirs(pooler_dir, exist_ok=True)
        torch.save(pool_emb, os.path.join(pooler_dir, f"batch_{batch_idx}.pt"))

    # ---------- housekeeping ----------
    del outputs, inputs, mean_emb
    if "pool_emb" in locals():
        del pool_emb
    gc.collect(); torch.cuda.empty_cache()

# =============================================================
# 4.  Summary
# =============================================================
if problem_batches:
    print("Found inf values in:", problem_batches)
print(f"All mean-pooled and pooler embeddings saved under {SAVE_DIR}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/41.1k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/1024 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/1024 [00:00<?, ?it/s]

Using device: cuda


...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

  0%|          | 0/118 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

All mean-pooled and pooler embeddings saved under simcse_mean_embeddings


In [ ]:
def load_layer_embeddings(layer_idx: int, num_layers = 13) -> torch.Tensor:
    """Return concatenated embeddings for `layer_idx`.
    The last index (== num_layers) denotes the pooler_output."""
    if layer_idx == num_layers:   # pooler
        layer_dir = os.path.join(SAVE_DIR, "pooler")
    else:
        layer_dir = os.path.join(SAVE_DIR, f"layer_{layer_idx}")

    batch_files = sorted(layer_dir for layer_dir in os.listdir(layer_dir))
    return torch.cat([torch.load(os.path.join(layer_dir, f)) for f in batch_files], dim=0)

In [ ]:
s = load_layer_embeddings(1)

In [ ]:
s.shape

torch.Size([30000, 768])

In [ ]:
for layer_idx in range(num_layers+1):
  a = load_layer_embeddings(layer_idx)
  if torch.isinf(a).sum() != 0:
    print(f"INF layer, {layer_idx}")
  if torch.isnan(a).sum() != 0:
    print(f"NANS layer, {layer_idx}")

In [ ]:
import logging
from datetime import datetime
import numpy as np
import torch
import gc

# === Настройка логгера ===
def setup_logging(log_file):
    logger = logging.getLogger("metrics_logger")
    logger.setLevel(logging.INFO)

    if logger.hasHandlers():
        logger.handlers.clear()

    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

    file_handler = logging.FileHandler(log_file, mode='w')
    file_handler.setFormatter(formatter)

    console_handler = logging.StreamHandler()
    console_handler.setFormatter(formatter)

    logger.addHandler(file_handler)
    logger.addHandler(console_handler)

    return logger

# === Подсчет метрики ===
def calculate_metric(a, b, metric_name, batch_size=1000, trials=10):
    cls = ALL_MEASURES[metric_name]
    metrics = []

    for i in range(trials):
        idx = np.random.permutation(a.shape[0])[:batch_size]
        metric = cls(a[idx], b[idx], shape='nd')
        metrics.append(metric)

    return np.mean(metrics), np.std(metrics)

In [ ]:
PATH = '/content/drive/MyDrive/PTLS/NLP/SIM-SCE/'
LOG_FILE = 'similarity_metrics.log'

In [ ]:
num_layers = 13
MODEL_NAME = "princeton-nlp/sup-simcse-roberta-base"  # *** SimCSE ***
SAVE_DIR   = "simcse_mean_embeddings"                 # ←- поменял имя директории

In [ ]:
# === Константы ===
METRICS = ['CKA']#, 'RSA', 'JaccardSimilarity', 'DistanceCorrelation']

# === Инициализация логгера ===
logger = setup_logging(LOG_FILE)

# === Основной процесс ===
try:
    logger.info("=== Start metrics calculation ===")
    logger.info(f"Metrics to compute: {', '.join(METRICS)}")
    start_time = datetime.now()

    for metric_name in METRICS:
        metric_start = datetime.now()
        logger.info(f"{'='*40}\nProcessing: {metric_name}")

        matrix = np.zeros((num_layers+1, num_layers+1))
        matrix_std = np.zeros((num_layers+1, num_layers+1))

        for i in range(num_layers+1):
            a = load_layer_embeddings(i)
            logger.info(f"Layer {i} | Shape: {a.shape}")

            for j in range(i, num_layers+1):
                b = load_layer_embeddings(j)

                try:
                    metr, metr_std = calculate_metric(a.float(), b.float(), metric_name, batch_size=5000, trials=5)
                    matrix[i, j] = matrix[j, i] = metr
                    matrix_std[i, j] = matrix_std[j, i] = metr_std

                    logger.info(f"{metric_name} | L{i}-L{j}: {metr:.4f} ± {metr_std:.4f} | "
                                f"GPU Mem: {torch.cuda.memory_allocated() / 1024**2:.1f} MB")
                except Exception as e:
                    logger.error(f"Metric error for L{i}-L{j}: {e}", exc_info=True)

                del b
                torch.cuda.empty_cache()
                gc.collect()
                clear_output()

            del a
            torch.cuda.empty_cache()
            gc.collect()

        save_res(matrix, f"{PATH}{metric_name}_meanpool.pkl")
        save_res(matrix_std, f"{PATH}{metric_name}_meanpool_std.pkl")

        logger.info(f"Finished {metric_name} in {datetime.now() - metric_start}")


except Exception as e:
    logger.critical(f"Fatal error: {e}", exc_info=True)
finally:
    logger.info(f"=== All metrics completed in {datetime.now() - start_time} ===")
    for handler in logger.handlers:
        handler.close()

2025-05-23 18:21:05,785 - INFO - Finished CKA in 0:01:14.676987
INFO:metrics_logger:Finished CKA in 0:01:14.676987
2025-05-23 18:21:05,786 - INFO - === All metrics completed in 0:01:14.678194 ===
INFO:metrics_logger:=== All metrics completed in 0:01:14.678194 ===
